In [505]:
%matplotlib inline
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np
import re 
import os 

In [506]:
filePath = os.path.join('data/DataAnalyst.csv')

In [507]:
df = pd.read_csv(filePath)

In [508]:
df.head(2)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1


In [509]:
# removed spaces and lower cased all column names to make them uniform
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [510]:
# removed unnecessary column 
df = df.drop('unnamed:_0',axis=1)

In [511]:
# inspect data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_title          2253 non-null   object 
 1   salary_estimate    2253 non-null   object 
 2   job_description    2253 non-null   object 
 3   rating             2253 non-null   float64
 4   company_name       2252 non-null   object 
 5   location           2253 non-null   object 
 6   headquarters       2253 non-null   object 
 7   size               2253 non-null   object 
 8   founded            2253 non-null   int64  
 9   type_of_ownership  2253 non-null   object 
 10  industry           2253 non-null   object 
 11  sector             2253 non-null   object 
 12  revenue            2253 non-null   object 
 13  competitors        2253 non-null   object 
 14  easy_apply         2253 non-null   object 
dtypes: float64(1), int64(1), object(13)
memory usage: 264.1+ KB


In [512]:
# pulled lower and upper salary ranges into their own columns
df['lower'] = df['salary_estimate'].apply(lambda x: pd.Series(re.findall(r'\d+',str(x).split('-')[0]),dtype='float64'))
df['upper'] = df['salary_estimate'].apply(lambda x: pd.Series(re.findall(r'\d+',str(x).split('-')[1]),dtype='float64'))

In [513]:
# split state and city
df['city'] = df['location'].apply(lambda x: x.split(',')[0].strip())
df['state'] = df['location'].apply(lambda x: x.split(',')[1].strip())
# fixed state abbreviation
df['state'] = df['state'].replace('Arapahoe','CO')

In [514]:
# removed ratings from company_name column
df['company_name'] = df['company_name'].map(lambda x: str(x).split('\n')[0])

In [515]:
# obtained the average salary expecation
df['avg_salary(K)'] =  df['upper'].add(df['lower']).div(2)

In [516]:
groupSalary = df.groupby('state', as_index=False).agg({'avg_salary(K)':['mean','max','min','count']})
groupSalary.columns = groupSalary.columns.map(' '.join).str.strip().str.replace(' ','_')
groupSalary['avg_salary(K)_mean'] = round(groupSalary['avg_salary(K)_mean'],1)
salaryData = groupSalary.sort_values(by='avg_salary(K)_mean', ascending=False).reset_index(drop=True)
salaryData

,state,avg_salary(K)_mean,avg_salary(K)_max,avg_salary(K)_min,avg_salary(K)_count
0,CA,88.4,150.0,40.0,626
1,IL,78.3,113.0,59.0,164
2,CO,73.5,91.0,62.0,96
3,NJ,73.0,106.0,39.5,86
4,NY,71.4,106.0,39.5,345
5,AZ,70.8,74.0,61.5,97
6,SC,68.3,76.0,64.5,3
7,NC,68.1,76.0,57.0,90
8,VA,65.2,80.5,56.0,48
9,WA,64.8,78.0,51.0,53


# Graph Salary Data

In [517]:
import plotly.express as px

fig = px.box(df[['state','avg_salary(K)']], x='state', y='avg_salary(K)',
                notched=True, title='USA Data Analyst Salaries')

fig.update_layout(
    yaxis_title = 'Mean Salary in $1000s',
    xaxis_title = 'US States'
)
fig.show()

In [518]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x = salaryData['state'],
        y = salaryData['avg_salary(K)_mean'],
        name = 'Average Salary',
        text = salaryData['avg_salary(K)_mean'],
        textposition = 'outside',
        textfont = dict(size=13, color='#1f77b4'),
        marker_color='rgb(158,202,225)', 
        marker_line_color='rgb(17, 69, 126)',
        marker_line_width=2, 
        opacity=0.7
    )
)
fig.add_trace(
    go.Scatter(
        x = salaryData['state'],
        y = salaryData['avg_salary(K)_min'],
        name = 'Min Salary',
    )
)
fig.add_trace(
    go.Scatter(
        x = salaryData['state'],
        y = salaryData['avg_salary(K)_max'],
        name = 'Max Salary',
    )
)
fig.update_traces(texttemplate='%{text:.2s}K')


In [519]:
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [520]:
mapData = dict(
    type='choropleth',
    locations = salaryData['state'].to_list(),
    locationmode = 'USA-states',
    z = salaryData['avg_salary(K)_mean'].to_list(),
    text = salaryData['avg_salary(K)_count'].to_list(),
    colorscale = 'Blues',
    colorbar = {'title': 'Avg Salary (1000s)'}
)
layout = dict(title = 'USA Data Analyst Average Salaries', 
                geo = dict(scope = 'usa',
                            showlakes = True,
                            lakecolor = 'rgb(85,173,240)'))

In [521]:
choromap = go.Figure(data = [mapData], layout = layout)

In [522]:
choromap.show()

In [523]:
df.head(3)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply,lower,upper,city,state,avg_salary(K)
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True,37.0,66.0,New York,NY,51.5
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1,37.0,66.0,New York,NY,51.5
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1,37.0,66.0,New York,NY,51.5


In [388]:
cols = ['avg_salary(K)','industry']

In [539]:
noMissingValues = df[df['industry']!='-1'].reset_index(drop=True)
grpDf = noMissingValues.groupby('industry', as_index=False).agg({'avg_salary(K)':'mean'})
grpDf.sort_values(by='avg_salary(K)', ascending=False).reset_index(drop=True)

,industry,avg_salary(K)
0,Drug & Health Stores,95.250000
1,Education Training Services,92.833333
2,Health Care Products Manufacturing,89.800000
3,Sports & Recreation,88.166667
4,Gambling,88.000000
...,...,...
83,Casual Restaurants,48.300000
84,Oil & Gas Services,46.375000
85,Grocery Stores & Supermarkets,41.500000
86,Trucking,38.500000
